**This tutorial was tested with the version `0.0.1-beta1` of NeuroTorch.**

<table class="nt-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://NeuroTorch.github.io/NeuroTorch/"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/neurotorch_logo_32px.png?raw=true" width=32px height=32px  />Documentation</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/NeuroTorch/NeuroTorch/blob/update-tutorials/tutorials/checkpoint_manager/save_and_load_tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/colab_logo_32px.png?raw=true" width=32px height=32px  />Run in Google Colab</a>
</td>
  <td>
    <a target="_blank" href="https://github.com/NeuroTorch/NeuroTorch/blob/main/tutorials/checkpoint_manager/save_and_load_tutorial.ipynb"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=32px height=32px />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/NeuroTorch/NeuroTorch/blob/main/tutorials/checkpoint_manager/save_and_load_tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/download_logo_32px.png?raw=true" width=32px height=32px />Download notebook</a>
  </td>
</table>

## Setup

Install the dependencies by running the following commands:

In [ ]:
#@title Install dependencies {display-mode: "form"}

RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

!pip install git+https://github.com/NeuroTorch/NeuroTorch.git
# !pip install neurotorch
!pip install pythonbasictools
!pip install unstable

The checkpoint manager is very useful in the NeuroTorch's training pipeline since it is a callback. Please note that it is possible to use the traditional pytorch save and load method, see [Pytorch save and load tutorial](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for more information.

As usual, the first thing is to import NeuroTorch

In [ ]:
import neurotorch as nt

First, simply create an object <code>CheckpointManager</code> with the name of your desired folder as an argument

In [ ]:
checkpoint_folder = f"./checkpoints/network"
checkpoint_manager = nt.CheckpointManager(checkpoint_folder)
print(checkpoint_manager)

The name of the checkpoint_folder must also be given to the <code>Sequential</code> in order to save the parameters of the network during training.

In [ ]:
network = nt.Sequential(layers=[nt.Linear(10, 10)], checkpoint_folder=checkpoint_folder).build()
print(network)

## What is in the checkpoint folder ?

After a training, you will obtain three different types of file.

### Training's parameters

These are your <code> .pth </code> files. Those files are the one that contain the parameter of your model at a certain time (at a certain iteration for instance). These are the files you might want to give to a colleague in order to reproduce your data.

### Network-checkpoint $\Rightarrow$ .Json summary

A json file will be generated which contain the name of your different training parameters that are saved. The best one is label in a way that you can easily get access to it later. This json is the bridge between your code and the <code>.pth</code>

### Training history figure

A training history is also generated to summarize the performance of your training. It can bring insight on how the loss evolve relative to the iteration or the learning rate. It is a great tool to compare results obtained with different hyperparameters.

Here's an example of the json file and the <code>.pth</code> :

In [ ]:
checkpoint_manager.checkpoints_meta_path

In [ ]:
checkpoint_manager.save_checkpoint(itr=0, itr_metrics={}, state_dict=network.state_dict())

In [ ]:
network.get_layer().forward_weights.data.zero_()

Once your parameters are saved, it is now time to load them up in ordre to "play" with your data. Simply call <code>load_checkpoint</code> and give it a <code>load_checkpoint_mode</code>. This last step will determine which of your multiple <code>.pth</code> will be loaded. For instance, one might want to load the last checkpoint (use <code>nt.LoadCheckpointMode.LAST_ITR</code>) or one might want to use the best one (nt.LoadCheckpointMode.BEST_ITR). Here is an example :

In [ ]:
network.load_checkpoint(checkpoints_meta_path=checkpoint_manager.checkpoints_meta_path, load_checkpoint_mode=nt.LoadCheckpointMode.LAST_ITR)

In [ ]:
network.get_layer().forward_weights

### A few more words on the checkpoint manager

The checkpoint manager built in NeuroTorch allows you to specify **when** you want to save the training's parameters. This is because saving the parameters can be a long process if it is done at each step. Also, it is generally not interesting to save the first iteration (the last one are generally the one you want)! With the checkpoint manger, you can save at a certain frequency or only saved the last iteration for example.

### Example from our tutorial *time_series_forecasting_wilson_cowan*

In this tutorial (that we highly recommend!), we use the checkpoint manager as a powerful tool during the training. If you inspect closely <code>main.py</code> of this tutorial, you will find the following <code>CheckpointManager</code> :

In [ ]:
checkpoint_folder = f"./checkpoints/network"
n_iterations = 1000
checkpoint_manager = nt.CheckpointManager(
	checkpoint_folder,
	metric="train_loss",
	minimise_metric=False,
	save_freq=-1,
	save_best_only=True,
	start_save_at=int(0.98 * n_iterations),
)
print(checkpoint_manager)

Let's look at every argument to make sure we truly understand what is happening here.
- First, we give the name of our checkpoint folder!
- <code>metric</code> : We give the name of the metric to collect the best checkpoint on
- <code>minimise_metric</code> : In this example, we wanted to maximise the metric. It is therefore set to False
- <code>save_freq</code> : Here, we absolutely want to save the last iteration. By specifying $-1$, we tell the checkpoint manager to save the last iteration no matter what.
- <code>save_best_only</code> : Not only do we want to save the last, we also want to save the best! This argument is therefore **True**
- <code>start_save_at</code> : We also want to save the iterations near the end of our training. Here, we start saving after 98% of our training is done

**Feel free to explore the different tutorials since most of them use the checkpoint manager!**